<a href="https://colab.research.google.com/github/Teodosiodg2002/practica-acuity/blob/main/PracticaAcuity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Acuity

# Celda 1: Importación de Librerías

In [46]:

import warnings
from google.colab import drive
warnings.filterwarnings("ignore", category=DeprecationWarning)
drive.mount('/content/drive')

import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado
from wurlitzer import sys_pipes
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Celda 2: Carga de Datos

In [47]:
# ==========================================
# CARGA DE DATASETS
# ==========================================
# Definimos la ruta donde se encuentran los archivos
from pathlib import Path
DATA_DIR = Path('/content/drive/MyDrive/datos/')

# Cargamos cada dataset
try:
    df_submission = pd.read_csv(DATA_DIR / 'submission.csv')
    print("submission.csv cargado")

    df_test_data = pd.read_csv(DATA_DIR / 'test_kaggle.csv')
    print("datos de test cargado")

    df_train_data = pd.read_csv(DATA_DIR / 'train_kaggle.csv')
    print("datos de entrenamiento cargado")

except FileNotFoundError as e:
    print(f"Error: Archivo no encontrado - {e}")
    print("Verifica que todos los archivos estén en la carpeta correcta.")
except Exception as e:
    print(f"Error inesperado al cargar archivos: {e}")




submission.csv cargado
datos de test cargado
datos de entrenamiento cargado


# CELDA 3: Limpieza Inicial y Selección


In [48]:
# Cargamos datos (asumiendo que df_train_data ya está cargado como en tu Celda 2)
df_clean = df_train_data.dropna(subset=['acuity']).copy()
y = df_clean['acuity']

# A. Limpieza específica de la columna 'pain' (Convertir a numérico)
def clean_pain(val):
    try:
        # Intentar convertir a float directo
        return float(val)
    except:
        # Manejar textos como 'unable', 'refused' -> Asignamos NaN para imputar luego
        return np.nan

df_clean['pain_num'] = df_clean['pain'].apply(clean_pain)

# B. Rellenar nulos en texto para evitar errores
df_clean['chiefcomplaint'] = df_clean['chiefcomplaint'].fillna('')

# C. Definir columnas
# NOTA: Ahora incluimos 'chiefcomplaint' y usamos 'pain_num'
cols_to_drop = [
    'stay_id', 'subject_id', 'hadm_id', 'subject_id_triage',
    'intime', 'outtime', 'acuity', 'pain' # Eliminamos pain original, usamos pain_num
]

X = df_clean.drop(columns=cols_to_drop, errors='ignore')

# CELDA 4: Split & Imputación (Ingeniería de Datos)


In [49]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Definición de grupos de columnas
numeric_features = ['temperature', 'o2sat', 'resprate', 'dbp', 'sbp', 'heartrate', 'pain_num']
text_features = 'chiefcomplaint'
# Identificar categóricas restantes automáticamente (excluyendo texto y numéricas)
categorical_features = [col for col in X.columns if col not in numeric_features and col != text_features]

print(f"Numéricas: {len(numeric_features)} | Categóricas: {len(categorical_features)} | Texto: 1")

# --- PIPELINES DE TRANSFORMACIÓN ---

# 1. Numérico: Imputación Mediana + Escalado (Ayuda a algunos modelos)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. Categórico: Imputación Constante + OneHot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 3. Texto: TF-IDF (Transforma texto en matriz numérica de importancia)
# max_features=100: Toma las 100 palabras más importantes (dolor, pecho, fiebre, etc.)
text_transformer = TfidfVectorizer(stop_words='english', max_features=100)

# --- UNIFICACIÓN (COLUMN TRANSFORMER) ---
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('txt', text_transformer, text_features)
    ],
    verbose_feature_names_out=False
)

# Aplicamos la transformación
# Esto genera arrays de numpy, ideal para el modelo
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# Convertimos a DataFrame solo para visualización/debug (opcional, consume memoria)
feature_names = (numeric_features +
                 list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out()) +
                 list(preprocessor.named_transformers_['txt'].get_feature_names_out()))

X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)

print(f"Dimensiones finales de X_train: {X_train_df.shape}")
# Ahora tienes ~150 columnas (vitals + onehot + 100 palabras clave)

Numéricas: 7 | Categóricas: 7 | Texto: 1
Dimensiones finales de X_train: (167240, 383)


# CELDA 5: Modelo Base (Random Forest)


In [50]:
# 1. Inicializar el modelo
# n_estimators=100: Crea 100 árboles de decisión
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
# class_weight='balanced': Ayuda al modelo a prestar atención a las clases minoritarias

# 2. Entrenar (Fit)
print("Entrenando modelo... (puede tardar unos segundos)")
rf_model.fit(X_train_processed, y_train)

# 3. Predecir en Validación
y_pred = rf_model.predict(X_val_processed)

# 4. Evaluación de Resultados
acc = accuracy_score(y_val, y_pred)
print(f"\n✅ Accuracy del modelo: {acc:.4f}")

print("\n📊 Reporte de Clasificación por Clase:")
print(classification_report(y_val, y_pred))

# (Opcional) Matriz de Confusión simple
print("\nMatriz de Confusión (Filas=Realidad, Col=Predicción):")
print(confusion_matrix(y_val, y_pred))

Entrenando modelo... (puede tardar unos segundos)

✅ Accuracy del modelo: 0.6762

📊 Reporte de Clasificación por Clase:
              precision    recall  f1-score   support

         1.0       0.78      0.53      0.63      2402
         2.0       0.66      0.56      0.60     13941
         3.0       0.68      0.84      0.75     22507
         4.0       0.58      0.09      0.15      2850
         5.0       1.00      0.01      0.02       110

    accuracy                           0.68     41810
   macro avg       0.74      0.41      0.43     41810
weighted avg       0.67      0.68      0.65     41810


Matriz de Confusión (Filas=Realidad, Col=Predicción):
[[ 1274   719   409     0     0]
 [  325  7798  5812     6     0]
 [   33  3361 18955   158     0]
 [    0    26  2581   243     0]
 [    0     1    93    15     1]]


# CELDA 6: PROCESAMIENTO TEST Y SUBMISSION

In [51]:
print("--- Procesando Test Set con Pipeline Mejorado ---")

# 1. Crear una copia para trabajar
df_test_clean = df_test_data.copy()

# ---------------------------------------------------------
# A. APLICAR LA MISMA LIMPIEZA MANUAL (Feature Engineering)
# ---------------------------------------------------------
# Es vital aplicar las mismas funciones 'custom' que definimos antes

# Limpieza de 'pain' (usando la función clean_pain definida en celdas anteriores)
df_test_clean['pain_num'] = df_test_clean['pain'].apply(clean_pain)

# Limpieza de 'chiefcomplaint' (rellenar nulos)
df_test_clean['chiefcomplaint'] = df_test_clean['chiefcomplaint'].fillna('')

# ---------------------------------------------------------
# B. PREPARAR X_test
# ---------------------------------------------------------

# Guardamos los IDs para el archivo final
submission_ids = df_test_clean['stay_id']

# Eliminamos las columnas que no usa el modelo (misma lista cols_to_drop)
# Nota: 'acuity' no existe en test, y 'pain' lo borramos porque ya creamos 'pain_num'
X_test = df_test_clean.drop(columns=cols_to_drop, errors='ignore')

# ---------------------------------------------------------
# C. TRANSFORMACIÓN (PIPELINE)
# ---------------------------------------------------------
print("Aplicando transformaciones (Pipeline)...")

# CRÍTICO: Usamos .transform(), NUNCA .fit() en datos de test.
# Esto asegura que el modelo reciba las columnas en el orden exacto que espera
# y use el mismo vocabulario para el texto.
try:
    X_test_processed = preprocessor.transform(X_test)
    print(f"Dimensiones de X_test procesado: {X_test_processed.shape}")
except Exception as e:
    print(f"Error en la transformación: {e}")
    print("Asegúrate de haber ejecutado la celda de entrenamiento (Fit) primero.")

# ---------------------------------------------------------
# D. PREDICCIÓN Y EXPORTACIÓN
# ---------------------------------------------------------

if 'rf_model' in locals():
    print("Generando predicciones...")

    # Predecir usando la matriz procesada
    predictions = rf_model.predict(X_test_processed)

    # Crear DataFrame final
    df_final = pd.DataFrame({
        'stay_id': submission_ids,
        'acuity': predictions
    })

    # Verificación rápida
    print("\nVista previa de predicciones:")
    print(df_final['acuity'].value_counts().sort_index())

    # Guardar a CSV
    output_filename = 'submission_mejorado.csv'
    df_final.to_csv(output_filename, index=False)
    print(f"\n✅ Archivo '{output_filename}' generado exitosamente.")

else:
    print("⚠️ El modelo 'rf_model' no está definido. Ejecuta la celda de entrenamiento primero.")

--- Procesando Test Set con Pipeline Mejorado ---
Aplicando transformaciones (Pipeline)...
Dimensiones de X_test procesado: (209050, 383)
Generando predicciones...

Vista previa de predicciones:
acuity
1.0      8009
2.0     59460
3.0    139406
4.0      2172
5.0         3
Name: count, dtype: int64

✅ Archivo 'submission_mejorado.csv' generado exitosamente.
